# SRIM Calculations for stopping power and energy losses

The calculations are based on SRIM stopping power tables.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

from srim import SR, Ion, Layer, Target

# directory where SRIM-2013 is located
srim_executable_directory = '/home/anton/.srim/'

## Extrapolations to transuranium elements

It is a sea of unknown to find a good method to extrapolate stopping powers for transuranium elements.

Some relevant sources:

* General comparison of different models on the marker: [CriticalReviewStoppingPowers](CriticalReviewStoppingPowers.pdf)
* Example of use of extrapolation model in superheavy element experiment: [Wittwer2010](Wittwer2010.pdf)
* Calculation of effective charge: [P. Sigmund, 1997](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.56.3781)
* Other ways to calculate the effective charge: [Hubert](nucl-data-tables-stopping.pdf) & [Northcliffe](nucl-data-tables-stopping_low.pdf)

The very basics can be explained as with the example of derivating stopping powers for No. By the use of the scaling relationship of the effective charge $q_{eff}$:
\begin{equation}
S_{No} = \frac{q_{eff, No}^2}{q_{eff, U}^2} S_U
\end{equation}
Where $ S_U $  and $S_{No} $ is the stopping power of uranium and nobelium respectively. As given in [P. Sigmund, 1997](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.56.3781) the effective charge is calculated as, 
\begin{align}
& q_{eff} = \lambda Z e \\
& \lambda = \sqrt{1-e^{-v_I/v_{TF}}}, \ v_{TF} = Z^{2/3}v_{0} = Z^{2/3} \alpha_{FS} c
\end{align}
where $ Z $ is the proton number, $ e $ the electronic charge, $ \alpha_{FS} $ the finestructure constant, $v_I$ is the ion velocity, and $ c $ the speed of light.  

Other models quantifies the effective charge differently, otherwise the scaling factor relationship can still be utilised.

#### Attenuation factor
According to the [review on the stopping power models](CriticalReviewStoppingPowers.pdf), and [Wittwer2010](Wittwer2010.pdf), SRIM overestimates the stopping power in general for low energy and heavy ions. Therefore it seems natural to further include an attenuation factor for the extrapolated stopping powers.

#### Target materials
Concerning target materials of transuranium elements, it should be sufficient with a linear scaling factor of the $Z$ of the reference element to the transuranic element of desire, see [Bethe-Bloch](https://en.wikipedia.org/wiki/Bethe_formula). I.e.

\begin{equation}
S_{No} = \frac{Z_{No}}{Z_U} S_U
\end{equation}

And here goes the implementation of the epSTF as introduced above.

In [ ]:
e_charge = 1.6021766e-19
alpha_FS = 1./137
c_vac = 299792458
#c_vac = c_vac/1.9605
u = 931.494
def GetepSTF(z_ion, z_ref, A, S_ref, E):
    v_TF_ion = z_ion**(2./3)*alpha_FS*c_vac
    v_TF_ref = z_ref**(2./3)*alpha_FS*c_vac
    v1 = np.sqrt(2*np.divide(E,A))
    l_ion = np.sqrt(1-np.exp(-np.divide(v1, v_TF_ion)))
    l_ref = np.sqrt(1-np.exp(-np.divide(v1, v_TF_ref)))
    q_eff_ion = l_ion*z_ion
    q_eff_ref = l_ref*z_ref
    factor = np.square(np.divide(q_eff_ion, q_eff_ref))
    S_ion = np.multiply(S_ref, factor)
    return S_ion, factor

## Test inputs - Ion and Stopping materials

In [ ]:
# Construct beam
A, E_per_A_max = 48, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
test_ion = Ion('Ca', energy=energy_max, mass=A)

# Al
density = 2.70e3 #mg/cm3 (from wikipedia)
# Construct a layer of Al, arbitrary width (as it is not relevant for these calcs)
test_layer = Layer({
    'Al': {
        'stoich': 1
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=1e-3*density, width=10000.0)

In [ ]:
test_layer.elements

## Class for the stopping power information

In [ ]:
class GenerateStoppingPower():
    
    def __init__(self, layer, ion, z_ion = -1, Z = -1, Z_ref = -1, **kwargs):
        #srim = SR(layer, ion, output_type=5, energy_min=1100) 
        #output_type = 5, means MeV / (mg/cm2)
        srim = SR(layer, ion, output_type=5, energy_min=1001) 
        results = srim.run(srim_executable_directory)

        self.energy_raw = 1e-3*results.data[0] #to MeV
        self.dEdX_raw = (results.data[1]+results.data[2])*(layer.density*1e3)*1e-4 # sum of electric and nuclear stopping power and convert to MeV per µm
        self.range_raw = results.data[3]
        #results.units
        
        if z_ion != -1:
            print("Extrapolating stopping power for ion with z=", z_ion)
            self.dEdX_raw, factor = GetepSTF(z_ion, ion.atomic_number, ion.mass, self.dEdX_raw, self.energy_raw)
            print("Mean scaling factor:", np.mean(factor))
            
        if Z != -1:
            print("Extrapolating stopping power for material with Z=", Z)
            factor = (float(Z)/Z_ref)
            self.dEdX_raw = factor*self.dEdX_raw
            print("Scaling factor:", factor)

        #interpolation of stopping power and range
        self.f_dEdX = interpolate.interp1d(self.energy_raw, self.dEdX_raw, kind='quadratic')
        self.energy_interp = np.arange(np.min(self.energy_raw), np.max(self.energy_raw), 10)
        self.f_range = interpolate.interp1d(self.energy_raw, self.range_raw, kind='quadratic')

        
        E_beam_0 = max(self.energy_raw) # MeV
        step_size = 0.1 #µm
        self.e_low_lim = 1.1e-3 # MeV

        e_loss = list()
        x_loss = list()

        e, x = [], []
        x_now = 0
        E_beam = E_beam_0
        e.append(E_beam)
        x.append(x_now)
        while E_beam > self.e_low_lim:
            #print("E_beam =",E_beam)
            x_now += step_size
            E_beam -= self.f_dEdX(E_beam)*step_size
            e.append(E_beam)
            x.append(x_now)
        self.x_loss = np.asarray(x)
        self.e_loss = np.asarray(e)
        self.f_eofx = (interpolate.interp1d(self.x_loss, self.e_loss, kind='quadratic'))
        self.f_xofe = (interpolate.interp1d(self.e_loss, self.x_loss, kind='quadratic'))  
        
        print("SRIM stopping power successfully calculated")
        print("\tCalculate energy loss as a function of traversed target material with function \'GetElossAfterX(E_init, x)'")
        print("\tCalculate traversed target material as a function of energy loss \'GetXAfterEloss(E_init, E_after)'")
        print("\tCalculate range in material as a function of energy \'GetRange(E_init)'")
        print("\tUnits: MeV, µm")

        
    def GetElossAfterX(self, E_init, x):
        x_start = self.f_xofe(E_init)
        #print("x_start=", x_start, "x_end=", x_start+x)
        return E_init - self.f_eofx(x_start+x)
    
    def GetXAfterEloss(self, E_init, E_after):
        x_start = self.f_xofe(E_init)
        return self.f_xofe(E_after) - x_start
    
    def GetRange(self, E_init, from_sr=False):
        if from_sr:
            return self.GetXAfterEloss(E_init, self.e_low_lim)
        else:
            return self.f_range(E_init)

In [ ]:
GSP = GenerateStoppingPower(layer=test_layer, ion=test_ion)

### Visualising test

In [ ]:
plt.plot(GSP.energy_raw, GSP.dEdX_raw, marker='o', label='dE/dx', linestyle='')
plt.plot(GSP.energy_interp, GSP.f_dEdX(GSP.energy_interp), label='Interpolation')
#plt.plot(energy, results.data[2], label='Sn')
plt.legend()

plt.xlabel('MeV')
plt.ylabel('MeV/µm')
plt.title("Stopping power")
#plt.xscale('log')
#plt.yscale('log')
#plt.xlim(1e-2, 1e2)
#plt.ylim(1, None)
plt.grid()
#plt.show()

In [ ]:
plt.plot(GSP.energy_raw, GSP.range_raw, marker='o', label='Range', linestyle='')
plt.plot(GSP.energy_interp, GSP.GetRange(GSP.energy_interp), label='Interpolation')
#plt.plot(energy, results.data[2], label='Sn')
plt.legend()

plt.xlabel('MeV')
plt.ylabel('µm')
plt.title("Range")
#plt.xscale('log')
#plt.yscale('log')
#plt.xlim(1e-2, 1e2)
#plt.ylim(1, None)
plt.grid()
#plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(GSP.x_loss, GSP.e_loss, marker='.', label='E(x)', linestyle='')
plt.plot(GSP.x_loss, GSP.f_eofx(GSP.x_loss), marker='', label='Interp. E(x)', linestyle='-')
plt.legend()

plt.xlabel('µm')
plt.ylabel('MeV')
plt.title('Stopping, E(x)')
#plt.xscale('log')
#plt.yscale('log')
#plt.xlim(1e-2, 1e2)
#plt.ylim(1, None)
plt.grid()
#plt.show()

In [ ]:
print("Successfully imported \'GenerateStoppingPower\'\n\tClass to Generate SRIM Stopping Power and calculate energy loss and traversed material.")